# Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
from pathlib import Path
import re
from collections import defaultdict
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass
from sklearn.metrics import precision_score, f1_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Preparation

## Retrieve annotations by 0.1 seconds

In [ ]:
file_name = "A_12_20130925_1212_01_2.txt"

In [ ]:
def load_annotations(file_name: str):
    df_raw = pd.read_csv(file_name, delimiter='\t', header=None)
    df = df_raw.iloc[:, [0, 2, 4, 6, 8]].copy()
    df.columns = ['Tier_Name', 'Start_Time', 'End_Time', 'Duration', 'Annotation_Value']
    return df

def filter_by_scene(df: pd.DataFrame):
    scene_records = df[df['Tier_Name'] == 'Scene']
    if scene_records.empty:
        print("Scene tier not found. Keeping all annotations.")
        return df

    # Retrieve the scene intervals
    scene_intervals = []
    for index, row in scene_records.iterrows():
        scene_intervals.append((row['Start_Time'], row['End_Time']))

    # Filter annotations based on these intervals
    df_filtered = pd.DataFrame()
    for start_time, end_time in scene_intervals:
        df_fragment = df[(df['Start_Time'] >= start_time) & (df['End_Time'] <= end_time)].copy()
        df_filtered = pd.concat([df_filtered, df_fragment])

    df = df_filtered.copy()
    return df

def time_string_to_ms(time_str: str):
    # Convert time string to milliseconds
    if pd.isna(time_str):
        return None
    h, m, s_ms = time_str.split(':')
    s, ms = s_ms.split('.')
    return int(h) * 3600000 + int(m) * 60000 + int(s) * 1000 + int(ms)

def preprocess_times(df: pd.DataFrame):
    df = df.copy()
    df['Start_Time_ms'] = df['Start_Time'].apply(time_string_to_ms)
    df['End_Time_ms'] = df['End_Time'].apply(time_string_to_ms)
    df['Start'] = (df['Start_Time_ms'] // 100).astype('Int64')
    df['End'] = (df['End_Time_ms'] // 100).astype('Int64')

    df = df.dropna()

    for c in ['Tier_Name', 'Annotation_Value']:
        df[c] = df[c].astype(str).str.strip()

    #Remove reference numbers in annotations, e.g. "14_mom spits the ball out of the mouth"
    df['Annotation_Value'] = df['Annotation_Value'].apply(lambda x: re.sub(r'^\d+_', '', x) if isinstance(x, str) else x)

    return df

def annotations_per_s(df: pd.DataFrame):
    rows = []
    max_t = int(df['End'].max())

    for t in range(0, max_t + 1):
        active_rows = df[(df['Start'] <= t) & (df['End'] > t)]
        if not active_rows.empty:
            for _, row in active_rows.iterrows():
              if row['Tier_Name'] not in ["Round", "Scene", "Comment"]:
                rows.append({'Time': t, 'Tier': row['Tier_Name'], 'Annotation': row['Annotation_Value']})

    return pd.DataFrame(rows)


def export(df: pd.DataFrame, output_name: str):
    df = df.dropna()
    df.to_csv(output_name, index=False)
    print(f"Per-second annotations saved to {Path(output_name).resolve()}")


def process_annotations(file_name: str, output_name: str = "annotations_per_t.csv"):
    df = load_annotations(file_name)
    df = filter_by_scene(df)
    df = preprocess_times(df)
    per_t_df = annotations_per_s(df)
    export(per_t_df, output_name)


process_annotations(file_name, "annotations_per_t.csv")


Per-second annotations saved to /content/drive/MyDrive/ISIR/RedBallGame/annotations_per_t.csv


## Annotations categorization

In [ ]:
df = pd.read_csv("annotations_per_t.csv")

### Facial@MOT

In [ ]:
# Categorize Facial@MOT annotations

facial_mot_counts = df[df['Tier']=='Facial@MOT'].value_counts('Annotation')
print(facial_mot_counts)

facial_mapping = {
    "smile showing teeth": "positive",
    "smile": "positive",
    "widely opened eyes": "positive",
    "O-shaped mouth": "neutral",
    "neutral": "neutral",
    "biting lower lip": "neutral",
    "invisible": "invisible"
}

def categorize_facial_mot(facial_annotation):
    return facial_mapping.get(facial_annotation, "neutral")

Annotation
invisible              1302
neutral                 503
smile showing teeth     436
widely opened eyes      208
smile                   197
O-shaped mouth          104
biting lower lip         18
Name: count, dtype: int64


### Utterance@MOT

In [ ]:
# Categorize Utterance@MOT annotations
utterance_mot_counts = df[df['Tier']=='Utterance@MOT'].value_counts('Annotation')
print(utterance_mot_counts)

# Utterance categories
questions = [
    "Ready?",
    "Can I please have a ball?",
    "What even set off you laughing so hard?"
]
commands = [
    "Drop it.",
    "Drop it in.",
    "Put it in here.",
    "Can you sit up?",
]
affection = [
    "I love you."
]

playful_sounds = [
    "chooga",
    "... tickle you",
    "Yeaaah!"
]

statements = [
    "She’s laughing so hard that she literally can’t even stay seated.",
    "I hope this is not the laugh before the cry.",
    "Really far.",
    "Too far.",
    "I take this one.",
    "This is harmonious to hear.",
    "Keep falling."
]

utterance_lists = {
    'question': questions,
    'statement': statements,
    'affection': affection,
    'playful_sounds': playful_sounds
}

utterance_category_mapping = {value: category for category, values in utterance_lists.items() for value in values}

Annotation
She’s laughing so hard that she literally can’t even stay seated.    66
This is harmonious to hear.                                          24
I hope this is not the laugh before the cry.                         22
What even set off you laughing so hard?                              22
Ready?                                                               18
... tickle you                                                       15
chooga                                                               15
Yeaaah!                                                              12
Keep falling.                                                        11
Can I please have a ball?                                             9
Really far.                                                           9
Drop it.                                                              9
I love you.                                                           8
Put it in here.                                      

In [ ]:
def categorize_utterance_mot(value):
    return utterance_category_mapping.get(value, "statement")

### Gaze

In [ ]:
# Categorize Gaze@MOT annotations
gaze_mot_counts = df[df['Tier']=='Gaze@MOT'].value_counts('Annotation')
print(gaze_mot_counts)

gaze_mot_mapping = {
    "child": "child",
    "invisible": "invisible",
    "ball": "main_object",
    "toys": "object",
    "into the distance": "distracted",
    "dad": "distracted",
    "floor": "distracted",
    "glass": "object",
    "toy": "object",
    "aside": "distracted",
}

def categorize_gaze_mot(gaze_annotation):
    return gaze_mot_mapping.get(gaze_annotation, "distracted") # Default to distracted if not in mapping

Annotation
invisible            1129
child                 880
ball                  296
toys                  162
into the distance      75
dad                    53
floor                  44
glass                  43
aside                  20
toy                    11
Name: count, dtype: int64


In [ ]:
# Categorize Gaze@CHI annotations
gaze_chi_counts = df[df['Tier']=='Gaze@CHI'].value_counts('Annotation')
print(gaze_chi_counts)

gaze_chi_mapping = {
    "mom": "mom",
    "mom's hand": "mom",
    "mom's legs": "mom",
    "ball": "main_object",
    "floor": "distracted",
    "toys": "object",
    "toy": "object",
    "glass": "object",
    "aside": "distracted",
    "ceiling": "distracted",
    "dad": "distracted"
}

def categorize_gaze_chi(gaze_annotation):
    return gaze_chi_mapping.get(gaze_annotation, "distracted") # Default to distracted if not in mapping

Annotation
mom           1637
ball           665
floor          276
toys           185
toy            147
glass          126
mom's hand      63
aside           24
ceiling         21
mom's legs      12
dad             11
Name: count, dtype: int64


### Action@MOT

In [ ]:
action_mot_counts = df[df['Tier']=='Action@MOT'].value_counts('Annotation')
print(action_mot_counts)

# df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('ball', 'main_object', regex=False)
# df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy pyramid', 'object', regex=False)
# df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('glass', 'object', regex=False)
# df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy', 'object', regex=False)

Annotation
holding the ball in mouth                       271
picking up the ball                             186
building a toy pyramid                          153
moving towards the ball to take it              111
putting the ball in mouth                       103
tossing the ball                                 84
taking the ball with mouth                       69
showing the glass                                51
extending a hand                                 51
spitting out the ball                            50
attracting dad's attention to the game           46
spitting the ball into the glass                 34
holding the glass                                28
taking the ball                                  22
tossing the toy                                  21
throwing the ball at the child                   18
picking up the glass                             16
tickling the child                               16
shaking the glass with the ball                  16
t

In [ ]:
def standardize_action_annotations(df: pd.DataFrame):
    df = df.copy() # Work on a copy to avoid modifying the original DataFrame in place
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('ball', 'main_object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy pyramid', 'object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('glass', 'object', regex=False)
    df.loc[df['Tier'] == 'Action@MOT', 'Annotation'] = df[df['Tier'] == 'Action@MOT']['Annotation'].str.replace('toy', 'object', regex=False)
    return df

In [ ]:
df = standardize_action_annotations(df)

### Applying categorization on df

In [ ]:
df['Annotation'] = df.apply(
    lambda row: categorize_facial_mot(row['Annotation']) if row['Tier'] == 'Facial@MOT' else
                categorize_utterance_mot(row['Annotation']) if row['Tier'] == 'Utterance@MOT' else
                categorize_gaze_mot(row['Annotation']) if row['Tier'] == 'Gaze@MOT' else
                categorize_gaze_chi(row['Annotation']) if row['Tier'] == 'Gaze@CHI' else
                row['Annotation'],
    axis=1
)

## Create objects

### Standard

In [ ]:
tiers = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI"
]

pivot_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

pivot_df = pivot_df[tiers]

pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

records = pivot_df.to_dict(orient='records')

X = records.copy()

### With GazeRelation

In [ ]:
tiersGR = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI",
    "GazeRelation"
]
pivot_gr_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

records_gr = pivot_gr_df.to_dict(orient='records')

for record in records_gr:
    mot_target = record.get("Gaze@MOT")
    chi_target = record.get("Gaze@CHI")

    if chi_target == "mom" and mot_target == "child":
        label = "MA"  # MutualAttention
    elif mot_target == "invisible":
        label = "<UNK>"  # Unknown
    elif chi_target == "mom":
        label = "SA(CHI)"  # SingleAttention(CHI)
    elif mot_target == "child":
        label = "SA(MOT)"  # SingleAttention(MOT)
    elif chi_target == mot_target and chi_target is not None:
        label = "ShA"  # SharedAttention
    elif chi_target != mot_target:
        label = "L"  # LostAttention
    else:
        label = "<UNK>"  # Unknown

    record["GazeRelation"] = label

pivot_gr_df = pd.DataFrame(records_gr)

pivot_gr_df = pivot_gr_df[tiersGR]

pivot_gr_df = pivot_gr_df.where(pd.notnull(pivot_gr_df), None)

records_gr = pivot_gr_df.to_dict(orient='records')

X_gr = records_gr.copy()

## Retrieve vocabs

In [ ]:
values = defaultdict(set)

for r in records_gr:
    for key in r.keys():
        value = r[key]
        if value is None or value == '' or value == 'invisible':
            value = "<UNK>"  # use <UNK> token for missing
        values[key].add(value)

vocab_action    = {token: idx for idx, token in enumerate(sorted(values['Action@MOT']))}
vocab_gaze      = {token: idx for idx, token in enumerate(sorted(values['Gaze@MOT']))}
vocab_utterance = {token: idx for idx, token in enumerate(sorted(values['Utterance@MOT']))}
vocab_prosody   = {token: idx for idx, token in enumerate(sorted(values['Prosody@MOT']))}
vocab_facial    = {token: idx for idx, token in enumerate(sorted(values['Facial@MOT']))}
vocab_laughter  = {token: idx for idx, token in enumerate(sorted(values['Laughter@MOT']))}
vocab_laughter_chi = {token: idx for idx, token in enumerate(sorted(values['Laughter@CHI']))}
vocab_gaze_chi  = {token: idx for idx, token in enumerate(sorted(values['Gaze@CHI']))}
vocab_gazerelation  = {token: idx for idx, token in enumerate(sorted(values['GazeRelation']))}

In [ ]:
vocabs = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
          "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
          "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

vocabs_gr = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
             "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
             "gazerelation": vocab_gazerelation, "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

# Helper functions

In [ ]:
def get_quality_metrics(preds, targets):
      accuracy = (np.array(preds) == np.array(targets)).mean()
      precision = precision_score(targets, preds, average='weighted', zero_division=0)
      f1 = f1_score(targets, preds, average='weighted', zero_division=0)
      return accuracy, precision, f1

In [ ]:
def get_data(isGazeRelation):
    if isGazeRelation:
        return X_gr, vocabs_gr
    else:
        return X, vocabs

In [ ]:
def get_loaders(X, isGazeRelation, isMixedPhases, train_size, val_size, n_parts):
    if not isMixedPhases:
        n_parts = 1

    X_train, y_train, X_val, y_val, X_test, y_test = DatasetSplit(dataset = X, n_parts = n_parts, train_size = train_size, val_size = val_size).main()

    train_batch_size = len(X_train) // n_parts
    val_batch_size = len(X_val) // n_parts
    test_batch_size = len(X_test) // n_parts

    # train_batch_size = len(X_train) // 1
    # val_batch_size = len(X_val) // 1
    # test_batch_size = len(X_test) // 1


    train_loader = get_loader(X_train, y_train, batch_size=train_batch_size, isGazeRelation=isGazeRelation)
    val_loader = get_loader(X_val, y_val, batch_size=val_batch_size, isGazeRelation=isGazeRelation)
    test_loader = get_loader(X_test, y_test, batch_size=test_batch_size, isGazeRelation=isGazeRelation)

    return train_loader, val_loader, test_loader

## Datasets

### Split dataset

In [ ]:
class DatasetSplit:
    def __init__(self, dataset, n_parts = 4, train_size = 0.5, val_size = 0.25):
          self.n_parts = n_parts
          self.train_size = train_size
          self.val_size = val_size

          self.dataset = dataset
          self.n = len(dataset)
          self.part_size = self.n // n_parts

    def split_dataset(self):
        train_idx, val_idx, test_idx = [], [], []

        for i in range(self.n_parts):
            start = i * self.part_size
            end = (i + 1) * self.part_size if i < self.n_parts - 1 else self.n

            part_indices = np.arange(start, end)

            n_train = int(len(part_indices) * self.train_size)
            n_val = int(len(part_indices) * self.val_size)

            train_idx.extend(part_indices[:n_train])
            val_idx.extend(part_indices[n_train:n_train + n_val])
            test_idx.extend(part_indices[n_train + n_val:])

        X_train = [X[i] for i in train_idx]
        X_validation = [X[i] for i in val_idx]
        X_test = [X[i] for i in test_idx]
        return X_train, X_validation, X_test

    def get_features_and_targets(self, dataset):
          X = []
          y = []

          for record in dataset:
              target = {}
              input = {}

              for key, value in record.items():
                  if key in ['Laughter@CHI', 'Gaze@CHI', 'GazeRelation']:
                      target[key] = value
                  else:
                      input[key] = value
              X.append(input)
              y.append(target)

          return X, y

    def main(self):
        train, validation, test = self.split_dataset()

        X_train, y_train = self.get_features_and_targets(train)
        X_validation, y_validation = self.get_features_and_targets(validation)
        X_test, y_test = self.get_features_and_targets(test)

        return X_train, y_train, X_validation, y_validation, X_test, y_test

### Data Loader (batches divsion)

In [ ]:
from torch.utils.data import Dataset

class SocialDataset(Dataset):
    def __init__(self, X, y):
        """
        X_list: list of sequences, each sequence is a list of dicts (time steps)
        y_list: list of targets (e.g., class indices)
        """
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def lookup(vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

def collate_fn(batch):
    # batch: list of (sequence_of_dicts, y_target_dict)
    sequences, targets_list = zip(*batch)

    # Process targets
    # Assuming targets_list is a list of dictionaries like {'Laughter@CHI': value, 'Gaze@CHI': value}
    # We need to convert these to tensors of indices
    target_laughter_indices = []
    target_gaze_indices = []

    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        # Use lookup for converting values to indices
        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    # Convert target indices to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    # Sequences (X_batch) are still lists of dictionaries.
    # Wrap each dictionary in a list to create sequences of length 1 for the model
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter)

def collate_fn_gr(batch):
    sequences, targets_list = zip(*batch)
    target_laughter_indices = []
    target_gaze_indices = []
    target_gazerelation_indices = []

    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        gazerelation_val = target_dict.get('GazeRelation')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))
        target_gazerelation_indices.append(lookup(vocab_gazerelation, gazerelation_val))

    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)
    targets_gazerelation = torch.tensor(target_gazerelation_indices, dtype=torch.long)

    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter, targets_gazerelation)

def get_loader(X, y, batch_size, isGazeRelation):
    dataset = SocialDataset(X, y)
    if isGazeRelation:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_gr)
    else:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return loader

## Evaluation Loop

In [ ]:
def evaluation_loop(model, loader, criterion, criterion_gaze, isGazeRelation):
    if isGazeRelation:
        return evaluation_on_set_gr(model, loader, criterion, criterion_gaze)
    else:
        return evaluation_on_set(model, loader, criterion, criterion_gaze)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_closest_gaze_annotation(predicted_embedding, gaze_embeddings_tensor, gaze_annotations):
    """Finds the closest gaze annotation for a given predicted embedding."""
    predicted_embedding_normalized = F.normalize(predicted_embedding.unsqueeze(0), dim=1)
    gaze_embeddings_tensor_normalized = F.normalize(gaze_embeddings_tensor, dim=1)

    # Calculate cosine similarity between the predicted embedding and all gaze embeddings
    similarity_scores = cosine_similarity(predicted_embedding_normalized, gaze_embeddings_tensor_normalized)

    # Convert similarity_scores to a PyTorch tensor
    similarity_scores_tensor = torch.tensor(similarity_scores)

    # Get the index of the highest similarity score
    closest_index = torch.argmax(similarity_scores_tensor).item()

    # Return the corresponding annotation
    return gaze_annotations[closest_index]

In [ ]:
def evaluation_on_set(model, loader, criterion, criterion_gaze):
        total_loss = 0
        targets_gazes = []
        preds_gazes = []
        targets_laughters = []
        preds_laughters = []

        for X_batch, (targets_gaze, targets_laughter) in loader:
            predicted_gaze, logits_laughter = model(X_batch, device)

            # Logits are already squeezed in the forward method
            similarity_target = torch.ones(predicted_gaze.size(0), device=device)
            # Get embeddings for target gaze annotations
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)
            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))
            total_loss += (loss_gaze + loss_laughter).item()


            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            gaze_chi_embeddings = model.sentence_encoder.encode(gaze_chi_annotations)
            gaze_chi_embeddings_tensor = torch.tensor(gaze_chi_embeddings).to(device)

            predicted_gaze_annotations = [
                get_closest_gaze_annotation(pred_embed, gaze_chi_embeddings_tensor, gaze_chi_annotations)
                for pred_embed in predicted_gaze
            ]

            # Convert predicted gaze annotations back to indices for metric calculation
            predicted_gaze_indices = [vocab_gaze_chi.get(anno, vocab_gaze_chi["<UNK>"]) for anno in predicted_gaze_annotations]

            targets_gazes.extend(targets_gaze.cpu().numpy())
            preds_gazes.extend(predicted_gaze_indices) # Use indices for predictions

            targets_laughters.extend(targets_laughter.cpu().numpy())
            preds_laughters.extend(torch.argmax(logits_laughter, dim=1).cpu().numpy())

        avg_loss = total_loss / len(loader)

        gaze_accuracy, gaze_precision, gaze_f1 = get_quality_metrics(preds_gazes, targets_gazes)
        laughter_accuracy, laughter_precision, laughter_f1 = get_quality_metrics(preds_laughters, targets_laughters)

        # Placeholder for gaze_sim, as it was removed
        gaze_sim = 0.0

        return avg_loss, loss_gaze, loss_laughter, gaze_accuracy, gaze_precision, gaze_f1, laughter_accuracy, laughter_precision, laughter_f1

In [ ]:
def evaluation_on_set_gr(model, loader, criterion, criterion_gaze):
        total_loss = 0
        targets_gazes = []
        preds_gazes = []
        targets_laughters = []
        preds_laughters = []
        targets_gazerelations = []
        preds_gazerelations = []

        for X_batch, (targets_gaze, targets_laughter, targets_gazerelation) in loader:
            predicted_gaze, logits_laughter, logits_gazerelation = model(X_batch, device)

            # Logits are already squeezed in the forward method
            similarity_target = torch.ones(predicted_gaze.size(0), device=device)
            # Get embeddings for target gaze annotations
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)
            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))
            loss_gazerelation = criterion(logits_gazerelation, targets_gazerelation.to(device))

            total_loss += (loss_gaze + loss_laughter + loss_gazerelation).item()

            # Reshape targets_gaze to be 2D for cosine_similarity
            targets_gaze_2d = targets_gaze.cpu().numpy().reshape(-1, 1)
            predicted_gaze_2d = predicted_gaze.detach().cpu().numpy()
            # Note: gaze_sim calculation was removed in the previous turn, keeping it commented out or removing as needed.
            # gaze_sim = cosine_similarity(predicted_gaze_2d, targets_gaze_2d).mean().item()


            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            gaze_chi_embeddings = model.sentence_encoder.encode(gaze_chi_annotations)
            gaze_chi_embeddings_tensor = torch.tensor(gaze_chi_embeddings).to(device)

            predicted_gaze_annotations = [
                get_closest_gaze_annotation(pred_embed, gaze_chi_embeddings_tensor, gaze_chi_annotations)
                for pred_embed in predicted_gaze
            ]

            # Convert predicted gaze annotations back to indices for metric calculation
            predicted_gaze_indices = [vocab_gaze_chi.get(anno, vocab_gaze_chi["<UNK>"]) for anno in predicted_gaze_annotations]


            targets_gazes.extend(targets_gaze.cpu().numpy())
            preds_gazes.extend(predicted_gaze_indices) # Use indices for predictions

            targets_laughters.extend(targets_laughter.cpu().numpy())
            preds_laughters.extend(torch.argmax(logits_laughter, dim=1).cpu().numpy())

            targets_gazerelations.extend(targets_gazerelation.cpu().numpy())
            preds_gazerelations.extend(torch.argmax(logits_gazerelation, dim=1).cpu().numpy())

        avg_loss = total_loss / len(loader)

        gaze_accuracy, gaze_precision, gaze_f1 = get_quality_metrics(preds_gazes, targets_gazes)
        laughter_accuracy, laughter_precision, laughter_f1 = get_quality_metrics(preds_laughters, targets_laughters)
        # accuracy_gazerelation, gazerelation_precision, gazerelation_f1 = get_quality_metrics(preds_gazerelations, targets_gazerelations)

        # Placeholder for gaze_sim, as it was removed
        gaze_sim = 0.0

        return avg_loss, loss_gaze, loss_laughter, gaze_accuracy, gaze_precision, gaze_f1, laughter_accuracy, laughter_precision, laughter_f1

## Training Loop

In [ ]:
def train_loop(model, loader, criterion, criterion_gaze, optimizer, isGazeRelation):
        if isGazeRelation:
            return train_loop_gr(model, loader, criterion, criterion_gaze, optimizer)
        else:
            return train_loop_standard(model, loader, criterion, criterion_gaze, optimizer)

In [ ]:
def train_loop_gr(model, loader, criterion, criterion_gaze, optimizer):
    total_loss = 0
    for X_batch, (targets_gaze, targets_laughter, targets_gazerelation) in loader:
        # Move targets to the correct device
        targets_gaze = targets_gaze.to(device)
        targets_laughter = targets_laughter.to(device)
        targets_gazerelation = targets_gazerelation.to(device)

        predicted_gaze, logits_laughter, logits_gazerelation = model(X_batch, device)

        similarity_target = torch.ones(predicted_gaze.size(0), device=device)

        gaze_chi_annotations = list(vocab_gaze_chi.keys())
        target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
        target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

        loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)

        loss_laughter = criterion(logits_laughter, targets_laughter)
        loss_gazerelation = criterion(logits_gazerelation, targets_gazerelation)

        loss = loss_gaze + loss_laughter + loss_gazerelation

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return model


In [ ]:
def train_loop_standard(model, loader, criterion, criterion_gaze, optimizer):
        # Training loop with DataLoader
        for X_batch, (targets_gaze, targets_laughter) in loader:
            targets_gaze = targets_gaze.to(device)
            targets_laughter = targets_laughter.to(device)

            predicted_gaze, logits_laughter = model(X_batch, device)

            similarity_target = torch.ones(predicted_gaze.size(0), device=device)

            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in targets_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            loss_gaze = criterion_gaze(predicted_gaze, target_gaze_embeddings, similarity_target)

            loss_laughter = criterion(logits_laughter, targets_laughter.to(device))

            # Combine losses
            loss = loss_gaze + loss_laughter

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        return model

## Training Model

In [ ]:
@dataclass
class DatasetParams:
    isGazeRelation: bool
    isMixedPhases: bool
    train_size: int
    val_size: int
    n_parts: int

@dataclass
class ModelParams:
    hidden_dim: int
    num_layers: int
    dropout: float
    input_size: int
    bidirectional: bool

@dataclass
class TrainParams:
    lr: float
    weight_decay: float
    num_epochs: int

In [ ]:
def training_model(model, dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    model = model(
        vocabs,
        dataset_params.isGazeRelation,
        hidden_dim = model_params.hidden_dim,
        num_layers =  model_params.num_layers,
        # Apply dropout only if num_layers > 1
        dropout = model_params.dropout if model_params.num_layers > 1 else 0.0,
        bidirectional = model_params.bidirectional,
        input_size = model_params.input_size
    ).to(device)

    train_loader, val_loader, test_loader = get_loaders(dataset, dataset_params.isGazeRelation, dataset_params.isMixedPhases, dataset_params.train_size, dataset_params.val_size, dataset_params.n_parts)

    optimizer = optim.Adam(model.parameters(), lr = train_params.lr, weight_decay = train_params.weight_decay)
    criterion = nn.CrossEntropyLoss()
    criterion_emb = nn.CosineEmbeddingLoss()
    cosine_similarity = nn.CosineSimilarity(dim=1)

    num_epochs = train_params.num_epochs

    print("Starting training...")
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        model.train()

        model = train_loop(model, train_loader, criterion, criterion_emb, optimizer, dataset_params.isGazeRelation)

        # Calculate training metrics (optional, can be time-consuming)
        model.eval()
        with torch.no_grad():
            avg_train_loss, loss_gaze, loss_laughter, train_gaze_accuracy, train_gaze_precision, train_gaze_f1, train_laughter_accuracy, train_laughter_precision, train_laughter_f1 = evaluation_loop(model, train_loader, criterion, criterion_emb, dataset_params.isGazeRelation)

        # Validation
        model.eval()
        with torch.no_grad():
            avg_val_loss, loss_gaze, loss_laughter, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(model, val_loader, criterion, criterion_emb, dataset_params.isGazeRelation)

        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"\tTrain Loss: {avg_train_loss:.4f},  Train Gaze Acc: {train_gaze_accuracy:.4f}, Train Gaze Prec: {train_gaze_precision:.4f}, Train Gaze F1: {train_gaze_f1:.4f}, Train Laughter Acc: {train_laughter_accuracy:.4f}, Train Laughter Prec: {train_laughter_precision:.4f}, Train Laughter F1: {train_laughter_f1:.4f}, "
              f"\n\tVal Loss: {avg_val_loss:.4f}, Val Gaze Acc: {val_gaze_accuracy:.4f}, Val Gaze Prec: {val_gaze_precision:.4f}, Val Gaze F1: {val_gaze_f1:.4f}, Val Laughter Acc: {val_laughter_accuracy:.4f}, Val Laughter Prec: {val_laughter_precision:.4f}, Val Laughter F1: {val_laughter_f1:.4f}"
              )

    print("Training finished.")

    # Evaluate on Test Set
    model.eval()
    with torch.no_grad():
        avg_test_loss, loss_gaze, loss_laughter, test_accuracy_gaze, test_gaze_precision, test_gaze_f1, test_accuracy_laughter, test_laughter_precision, test_laughter_f1 = evaluation_loop(model, test_loader, criterion, criterion_emb, dataset_params.isGazeRelation)


    print("\nTest Set Evaluation:")
    print(f"Test Loss: {avg_test_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Gaze Prec: {test_gaze_precision:.4f}, Test Gaze F1: {test_gaze_f1:.4f}, Test Laughter Acc: {test_accuracy_laughter:.4f}, Test Laughter Prec: {test_laughter_precision:.4f}, Test Laughter F1: {test_laughter_f1:.4f}")

# Single LSTM (with Gaze Embedding)

## Model

In [ ]:
class EnfantReactionPredictor(nn.Module):
    def __init__(self,
                 vocabs,
                 isGazeRelation,
                 hidden_dim = 16,
                 num_layers = 2,
                 dropout = 0.0,
                 input_size = 40,
                 bidirectional = False
                 ):
        super().__init__()

        self.isGazeRelation = isGazeRelation

        # Vocabs
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # Embedding layers
        self.embed_gaze = nn.Embedding(len(vocab_gaze), 8)
        self.embed_utterance = nn.Embedding(len(vocab_utterance), 8)
        self.embed_prosody = nn.Embedding(len(vocab_prosody), 4)
        self.embed_facial = nn.Embedding(len(vocab_facial), 8)
        self.embed_laughter = nn.Embedding(len(vocab_laughter), 4)

        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        self.embed_laughter_chi = nn.Embedding(len(vocab_laughter_chi), 4)

        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # LSTM
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        self.dropout = nn.Dropout(dropout)

        # Determine the input size for the decoder based on bidirectionality
        decoder_input_dim = hidden_dim * (2 if bidirectional else 1)

        # Decoder to predict child gaze, laughter and mother's action
        self.decoder_laughter = nn.Linear(decoder_input_dim, len(vocab_laughter_chi))
        self.project_gaze = nn.Linear(decoder_input_dim, self.sentence_encoder.get_sentence_embedding_dimension())

        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(decoder_input_dim, len(self.vocab_gazerelation))

    # method for finding value's index in list
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # converting inputs into indices
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        all_seqs = []

        for sequence in batch_sequence_of_dicts:
            seq = []

            for step_dict in sequence:
                gaze = torch.tensor([self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT"))], dtype=torch.long)
                utterance = torch.tensor([self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT"))], dtype=torch.long)
                prosody = torch.tensor([self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT"))], dtype=torch.long)
                facial = torch.tensor([self.lookup(self.vocab_facial, step_dict.get("Facial@MOT"))], dtype=torch.long)
                laughter = torch.tensor([self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT"))], dtype=torch.long)

                action_text = step_dict.get("Action@MOT") or "<UNK>"
                encoded_action = torch.tensor(self.sentence_encoder.encode(action_text), dtype=torch.float32).unsqueeze(0)

                step_emb = torch.cat([
                    self.embed_gaze(gaze),
                    self.embed_utterance(utterance),
                    self.embed_prosody(prosody),
                    self.embed_facial(facial),
                    self.embed_laughter(laughter),
                    encoded_action
                ], dim=-1)  # shape: [1, D_total]

                seq.append(step_emb)

            seq = torch.cat(seq, dim=0)  # shape: [T, D_total]
            all_seqs.append(seq)

        batch_tensor = torch.stack(all_seqs).to(device)  # shape: [B, T, D_total]
        return batch_tensor


    def forward(self, batch_sequence_of_dicts, device):
        # Collect encoded steps and targets
        encoded_steps = self.encode_input_batch(batch_sequence_of_dicts, device)


        # Run the LSTM over the full sequence
        output, (h_n, c_n) = self.lstm(encoded_steps)         # output: [batch_size, 1, hidden_dim]

        # Apply decoders to the output of the LSTM for the single time step
        # Squeeze to remove the sequence length dimension of size 1
        logits_laughter = self.decoder_laughter(output.squeeze(1)) # [batch_size, num_laughter]
        predicted_gaze_embeddings = self.project_gaze(output.squeeze(1))

        if self.isGazeRelation:
            logits_gazerelation = self.decoder_gazerelation(output.squeeze(1)) # [batch_size, num_gazerelation]
            return (predicted_gaze_embeddings, logits_laughter, logits_gazerelation)

        return predicted_gaze_embeddings, logits_laughter

## Grid Search

In [ ]:
from sklearn.model_selection import ParameterGrid

def grid_search(model, dataset_params: DatasetParams, param_grid: dict):
    best_val_loss = float('inf')
    best_params = None
    results = []

    # Assuming vocabs are consistent across different runs for now
    # If vocabs depend on dataset_params, need to regenerate them here
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    param_grid = ParameterGrid(param_grid)

    print(f"Starting Grid Search with {len(param_grid)} combinations...")

    for i, params in enumerate(tqdm(param_grid, desc="Processing params", unit="set")):
        print(f"\n--- Running combination {i+1}/{len(param_grid)} ---")
        print(f"Parameters: {params}")

        # Create ModelParams and TrainParams dataclasses from the current combination
        model_params = ModelParams(
            hidden_dim=params['hidden_dim'],
            num_layers=params['num_layers'],
            dropout=params['dropout'],
            bidirectional=params['bidirectional'],
            input_size=416
        )
        train_params = TrainParams(
            lr=params['lr'],
            weight_decay=params['weight_decay'],
            num_epochs=params['num_epochs']
        )

        # Initialize and train the model
        model_instance = model(
            vocabs,
            dataset_params.isGazeRelation,
            hidden_dim=model_params.hidden_dim,
            num_layers=model_params.num_layers,
            dropout=model_params.dropout,
            bidirectional=model_params.bidirectional,
            input_size=model_params.input_size
        ).to(device)

        train_loader, val_loader, test_loader = get_loaders(
            dataset,
            dataset_params.isGazeRelation,
            dataset_params.isMixedPhases,
            dataset_params.train_size,
            dataset_params.val_size,
            dataset_params.n_parts
        )

        optimizer = optim.Adam(model_instance.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
        criterion = nn.CrossEntropyLoss()
        criterion_gaze = nn.CosineEmbeddingLoss()

        for epoch in tqdm(range(train_params.num_epochs), desc=f"Training Epochs (Params {params})"):
             model_instance.train()
             model_instance = train_loop(model_instance, train_loader, criterion, criterion_gaze, optimizer, dataset_params.isGazeRelation)

             model_instance.eval()
             with torch.no_grad():
                avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(model_instance, val_loader, criterion, criterion_gaze, dataset_params.isGazeRelation)


        # Evaluate on validation set after training
        model_instance.eval()
        with torch.no_grad():
            avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(model_instance, test_loader, criterion, criterion_gaze, dataset_params.isGazeRelation)

        print(f"Validation Loss for this combo: {avg_val_loss:.4f}")

        # Store results
        results.append({
            'params': params,
            'val_loss': avg_val_loss,
            'val_gaze_accuracy': val_gaze_accuracy,
            'val_gaze_precision': val_gaze_precision,
            'val_gaze_f1': val_gaze_f1,
            'val_laughter_accuracy': val_laughter_accuracy,
            'val_laughter_precision': val_laughter_precision,
            'val_laughter_f1': val_laughter_f1,
        })

        # Check for best model based on validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_params = params
            # Optionally save the best model state_dict here
            # torch.save(model_instance.state_dict(), 'best_model_state_dict.pth')

    print("\n--- Grid Search Finished ---")
    print(f"Best parameters found: {best_params}")
    print(f"Best validation loss: {best_val_loss:.4f}")

    # You can return results or the best parameters/model
    return results, best_params


### Results - Standard

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results:
      print(res)

results_df = pd.DataFrame(results)

print("\nGrid Search Results DataFrame:")
display(results_df)

results_df.to_csv('grid_search_results_gaze_emb_results_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8569

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8236

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.7979

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.801650881767273, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.8737980769230769), 'val_laughter_precision': 0.8587648742386645, 'val_laughter_f1': 0.8655316804997005}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.8582247495651245, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_g

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.801651,0.210337,0.044241,0.073106,0.873798,0.858765,0.865532
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.858225,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.853188,0.210337,0.044241,0.073106,0.724760,0.866623,0.775746
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.958511,0.210337,0.044241,0.073106,0.736779,0.849700,0.781331
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.833641,0.210337,0.044241,0.073106,0.822115,0.849104,0.834548
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.889538,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.838621,0.210337,0.044241,0.073106,0.752404,0.870705,0.795792
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.907704,0.210337,0.044241,0.073106,0.811298,0.880812,0.837925
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.817913,0.210337,0.044241,0.073106,0.750000,0.851839,0.790597
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.887178,0.210337,0.044241,0.073106,0.831731,0.885133,0.852562


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results2:
      print(res)

results_df_2 = pd.DataFrame(results2)

print("\nGrid Search Results DataFrame:")
display(results_df_2)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.1494

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.1692

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.1494

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.3589394092559814, 'val_gaze_accuracy': np.float64(0.49603174603174605), 'val_gaze_precision': 0.24604749307130258, 'val_gaze_f1': 0.328933518588691, 'val_laughter_accuracy': np.float64(0.3630952380952381), 'val_laughter_precision': 0.13480182492158538, 'val_laughter_f1': 0.19661052193803286}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.4693374633789062, 'val_gaze_accuracy': np.float64(0.49603174603174605), 'val_gaze_precision': 0.24604749307130258, 'val_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.358939,0.496032,0.246047,0.328934,0.363095,0.134802,0.196611
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.469337,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.312282,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.236566,0.496032,0.246047,0.328934,0.478175,0.698230,0.419939
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.382076,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.538490,0.496032,0.246047,0.328934,0.369048,0.136196,0.198965
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.389385,0.480159,0.294938,0.365418,0.490079,0.586616,0.479897
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.312604,0.496032,0.246047,0.328934,0.442460,0.457813,0.448965
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.360281,0.496032,0.246047,0.328934,0.432540,0.626695,0.355077
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.212256,0.496032,0.246047,0.328934,0.462302,0.482432,0.470016


In [ ]:
results_df_2.to_csv('grid_search_results_gaze_emb_results_df_2.csv', index=False)

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results3:
      print(res)

results_df_3 = pd.DataFrame(results3)

print("\nGrid Search Results DataFrame:")
display(results_df_3)

results_df_3.to_csv('grid_search_results_gaze_emb_results_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8782

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.9219

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.8718

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9076759252283308, 'val_gaze_accuracy': np.float64(0.44594594594594594), 'val_gaze_precision': 0.35878275646592345, 'val_gaze_f1': 0.3854638375617886, 'val_laughter_accuracy': np.float64(0.7132132132132132), 'val_laughter_precision': 0.7019806348083705, 'val_laughter_f1': 0.706191631622587}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9416055348184373, 'val_gaze_accuracy': np.float64(0.42492492492492495), 'val_gaze_precision': 0.3545594906268787, 'val_ga

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.907676,0.445946,0.358783,0.385464,0.713213,0.701981,0.706192
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.941606,0.424925,0.354559,0.372835,0.704204,0.726774,0.712076
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.899551,0.426426,0.342477,0.367993,0.713213,0.701981,0.706192
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.896247,0.435435,0.339908,0.371173,0.714715,0.727252,0.719736
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.902905,0.429429,0.337738,0.367096,0.726727,0.713455,0.717554
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.910424,0.442943,0.367474,0.386455,0.681682,0.691226,0.685858
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.871849,0.417417,0.334074,0.359095,0.714715,0.703922,0.708005
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.925605,0.421922,0.364168,0.373057,0.704204,0.726774,0.712076
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.888558,0.429429,0.339947,0.368157,0.725225,0.711529,0.715712
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.910031,0.439940,0.356992,0.381951,0.692192,0.706910,0.698094


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results4:
      print(res)

results_df_4 = pd.DataFrame(results4)

print("\nGrid Search Results DataFrame:")
display(results_df_4)

results_df_4.to_csv('grid_search_results_gaze_emb_results_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.9653

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.9509

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.9252

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9392092724641165, 'val_gaze_accuracy': np.float64(0.42092746730083236), 'val_gaze_precision': 0.5044067632203383, 'val_gaze_f1': 0.4338709750954731, 'val_laughter_accuracy': np.float64(0.6979785969084423), 'val_laughter_precision': 0.6929560538164926, 'val_laughter_f1': 0.6949804954408726}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0015367070833843, 'val_gaze_accuracy': np.float64(0.5089179548156956), 'val_gaze_precision': 0.3925791167423469, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.939209,0.420927,0.504407,0.433871,0.697979,0.692956,0.694980
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.001537,0.508918,0.392579,0.432080,0.684899,0.702777,0.690534
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.951655,0.422117,0.507498,0.434759,0.676576,0.667061,0.670261
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.982810,0.514863,0.403268,0.440032,0.674197,0.674732,0.674461
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.944131,0.414982,0.495519,0.424487,0.684899,0.681203,0.682832
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.013973,0.529132,0.402903,0.448400,0.633769,0.646375,0.638686
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.925204,0.439952,0.492816,0.443003,0.703924,0.694845,0.697280
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.978998,0.391201,0.436797,0.389809,0.671819,0.674005,0.672852
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.953920,0.445898,0.498878,0.448806,0.656361,0.646647,0.650235
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.002704,0.386445,0.434954,0.386630,0.664685,0.668635,0.666479


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results5:
      print(res)

results_df_5 = pd.DataFrame(results5)

print("\nGrid Search Results DataFrame:")
display(results_df_5)

results_df_5.to_csv('grid_search_results_gaze_emb_results_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8120

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8140

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.7832

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.7999229316528027, 'val_gaze_accuracy': np.float64(0.4482758620689655), 'val_gaze_precision': 0.23489907885051511, 'val_gaze_f1': 0.29261799729165805, 'val_laughter_accuracy': np.float64(0.7835909631391201), 'val_laughter_precision': 0.7809281613766356, 'val_laughter_f1': 0.7817905438739964}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.7968397438526154, 'val_gaze_accuracy': np.float64(0.44589774078478), 'val_gaze_precision': 0.22922589562955484, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.799923,0.448276,0.234899,0.292618,0.783591,0.780928,0.781791
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.796840,0.445898,0.229226,0.291270,0.802616,0.803916,0.803191
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.810950,0.460166,0.356368,0.391168,0.784780,0.781112,0.781646
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.792413,0.468490,0.364618,0.398080,0.809750,0.809469,0.809605
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.796840,0.463734,0.358044,0.393940,0.802616,0.799405,0.799022
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.783204,0.442331,0.227052,0.289602,0.783591,0.781138,0.781985
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.798494,0.455410,0.385069,0.392601,0.793103,0.790126,0.790804
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.815516,0.468490,0.363512,0.396998,0.804994,0.805296,0.805140
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.808086,0.463734,0.358044,0.393940,0.790725,0.787125,0.787356
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.821871,0.463734,0.356940,0.392877,0.799049,0.799520,0.799274


### Results - with Gaze Relation

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results_gr, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr:
      print(res)

results_gr_df = pd.DataFrame(results_gr)

print("\nGrid Search Results DataFrame:")
display(results_gr_df)

results_gr_df.to_csv('grid_search_results_gaze_emb_results_gr_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.9521

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.7706

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 0.7706

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.0475231409072876, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.9002403846153846), 'val_laughter_precision': 0.8104327500924556, 'val_laughter_f1': 0.8529791879530968}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.0424246788024902, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.047523,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.042425,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.955987,0.210337,0.044241,0.073106,0.770433,0.830805,0.797525
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.988812,0.210337,0.044241,0.073106,0.806490,0.862143,0.829782
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.203815,0.210337,0.044241,0.073106,0.862981,0.853815,0.858187
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.048010,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.968240,0.210337,0.044241,0.073106,0.826923,0.866712,0.843967
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.160146,0.210337,0.044241,0.073106,0.129808,0.910501,0.076762
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.956394,0.210337,0.044241,0.073106,0.715144,0.846311,0.766026
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.022789,0.210337,0.044241,0.073106,0.716346,0.865426,0.769584


In [ ]:
results_gr_df.to_csv('grid_search_results_gaze_emb_results_gr_df.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results_gr2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr2:
      print(res)

results_gr_df_2 = pd.DataFrame(results_gr2)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_2)

results_gr_df_2.to_csv('grid_search_results_gaze_emb_results_gr_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2480

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 1.2552

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 1.2236

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 1.2841920852661133, 'val_gaze_accuracy': np.float64(0.4523809523809524), 'val_gaze_precision': 0.30649116014969674, 'val_gaze_f1': 0.36541272405569664, 'val_laughter_accuracy': np.float64(0.5535714285714286), 'val_laughter_precision': 0.5993645941977508, 'val_laughter_f1': 0.5606541631374142}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 1.3164284080266953, 'val_gaze_accuracy': np.float64(0.47619047619047616), 'val_gaze_precision': 0.31830914183855363, 'val_g

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.284192,0.452381,0.306491,0.365413,0.553571,0.599365,0.560654
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.316428,0.476190,0.318309,0.381563,0.527778,0.554564,0.535711
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.312604,0.452381,0.303979,0.363408,0.537698,0.588095,0.544339
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",1.285877,0.476190,0.312461,0.377330,0.468254,0.485745,0.475227
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.292241,0.452381,0.306491,0.365413,0.557540,0.602152,0.564691
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.342770,0.452381,0.312418,0.369592,0.470238,0.491239,0.478069
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.274548,0.500000,0.335852,0.398328,0.531746,0.615241,0.530219
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",1.286195,0.474206,0.318686,0.380981,0.474206,0.495141,0.481979
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.358213,0.454365,0.307003,0.366423,0.478175,0.533741,0.483683
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",1.273557,0.458333,0.315606,0.371097,0.498016,0.520580,0.505822


In [ ]:
results_gr_df_2.to_csv('grid_search_results_gaze_emb_results_gr_df_2.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results_gr3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr3:
      print(res)

results_gr_df_3 = pd.DataFrame(results_gr3)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_3)

results_gr_df_3.to_csv('grid_search_results_gaze_emb_results_gr_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8848

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 0.8927

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 0.8776

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 0.9075093931621976, 'val_gaze_accuracy': np.float64(0.42342342342342343), 'val_gaze_precision': 0.3435910236396854, 'val_gaze_f1': 0.3667251027243027, 'val_laughter_accuracy': np.float64(0.7087087087087087), 'val_laughter_precision': 0.6983360349333422, 'val_laughter_f1': 0.7024105186267349}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 0.9479888147777982, 'val_gaze_accuracy': np.float64(0.36786786786786785), 'val_gaze_precision': 0.28442788029883176, 'val_ga

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.907509,0.423423,0.343591,0.366725,0.708709,0.698336,0.702411
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.947989,0.367868,0.284428,0.306793,0.657658,0.687114,0.667972
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.899491,0.436937,0.347195,0.373729,0.672673,0.676520,0.674503
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",0.961735,0.391892,0.300136,0.324631,0.690691,0.730492,0.701926
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.930327,0.432432,0.352267,0.375723,0.711712,0.704488,0.707567
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.953502,0.367868,0.284428,0.306793,0.725225,0.744864,0.732067
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.900335,0.462462,0.376936,0.400335,0.734234,0.722997,0.726679
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",0.978834,0.426426,0.363815,0.378710,0.659159,0.713623,0.672987
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.877559,0.420420,0.348861,0.367883,0.749249,0.742903,0.745425
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",0.950196,0.379880,0.289018,0.315004,0.648649,0.670699,0.657241


In [ ]:
results_gr_df_3.to_csv('grid_search_results_gaze_emb_results_gr_df_3.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results_gr4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr4:
      print(res)

results_gr_df_4 = pd.DataFrame(results_gr4)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_4)

results_gr_df_4.to_csv('grid_search_results_gaze_emb_results_gr_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Validation Loss for this combo: 0.9994

--- Running combination 3/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 0.9459

--- Running combination 4/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs':…

In [ ]:
results_gr_df_4.to_csv('grid_search_results_gaze_emb_results_gr_df_4.csv', index=False)

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results_gr5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr5:
      print(res)

results_gr_df_5 = pd.DataFrame(results_gr5)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_5)

results_gr_df_5.to_csv('grid_search_results_gaze_emb_results_gr_df_5.csv', index=False)

In [ ]:
results_gr_df_5.to_csv('grid_search_results_gaze_emb_results_gr_df_5.csv', index=False)

## Training Best Models

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=False,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

model2 = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Starting training...


Training Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], 	Train Loss: 0.7902,  Train Gaze Acc: 0.5649, Train Gaze Prec: 0.3191, Train Gaze F1: 0.4078, Train Laughter Acc: 0.7435, Train Laughter Prec: 0.7471, Train Laughter F1: 0.7451, 
	Val Loss: 0.8806, Val Gaze Acc: 0.4819, Val Gaze Prec: 0.2322, Val Gaze F1: 0.3134, Val Laughter Acc: 0.7089, Val Laughter Prec: 0.7139, Val Laughter F1: 0.7108
Epoch [2/25], 	Train Loss: 0.7089,  Train Gaze Acc: 0.5649, Train Gaze Prec: 0.3632, Train Gaze F1: 0.4212, Train Laughter Acc: 0.7151, Train Laughter Prec: 0.7458, Train Laughter F1: 0.7229, 
	Val Loss: 0.8610, Val Gaze Acc: 0.4819, Val Gaze Prec: 0.2322, Val Gaze F1: 0.3134, Val Laughter Acc: 0.6800, Val Laughter Prec: 0.6948, Val Laughter F1: 0.6837
Epoch [3/25], 	Train Loss: 0.6790,  Train Gaze Acc: 0.5969, Train Gaze Prec: 0.5630, Train Gaze F1: 0.5537, Train Laughter Acc: 0.7411, Train Laughter Prec: 0.7430, Train Laughter F1: 0.7420, 
	Val Loss: 0.8283, Val Gaze Acc: 0.5531, Val Gaze Prec: 0.4433, Val Gaze F1: 0.4709, Val Laughter

### 10-fold CV

In [ ]:
def get_targets(y):
    target_laughter_indices = []
    target_gaze_indices = []

    for target_dict in y:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        # Use lookup for converting values to indices
        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    return targets_gaze, targets_laughter

In [ ]:
from sklearn.model_selection import KFold

def CrossValidation(dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    # Combine train and test data for cross-validation
    X_combined, vocabs = get_data(dataset_params.isGazeRelation)
    # Initialize KFold
    kf = KFold(n_splits=10, shuffle=False)

    test_losses = []
    test_gaze_accuracies = []
    test_laughter_accuracies = []

    print("Starting 10-Fold Cross-Validation...")

    # Iterate over each fold
    for fold, (train_index, test_index) in tqdm(enumerate(kf.split(X_combined)), total=kf.get_n_splits(), desc="Cross-Validation Progress"):
        print(f"\n--- Fold {fold+1}/10 ---")

        # Split data into train and test sets for the current fold
        train_fold = [X_combined[i] for i in train_index]
        test_fold = [X_combined[i] for i in test_index]

        X_train_fold, y_train_fold = DatasetSplit(dataset = X, n_parts = 4, train_size = 0.7, val_size = 0).get_features_and_targets(train_fold)
        X_test_fold, y_test_fold = DatasetSplit(dataset = X, n_parts = 4, train_size = 0.7, val_size = 0).get_features_and_targets(test_fold)
        y_train_fold_gaze, y_train_fold_laughter = get_targets(y_train_fold)
        y_test_fold_gaze, y_test_fold_laughter = get_targets(y_test_fold)

        # Initialize a new model for each fold to ensure independent training
        model = EnfantReactionPredictor(
            vocabs,
            dataset_params.isGazeRelation,
            model_params.hidden_dim,
            model_params.num_layers,
            model_params.dropout,
            model_params.input_size,
            model_params.bidirectional
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()
        criterion_gaze = nn.CosineEmbeddingLoss()

        num_epochs_fold = 10
        for epoch in range(num_epochs_fold):
            model.train()
            (predicted_gaze, logits_laughter) = model([X_train_fold], device)

            similarity_target = torch.ones(predicted_gaze.size(0), device=device)
            # Get embeddings for target gaze annotations
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in y_train_fold_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            # Remove the extra dimension from predicted_gaze
            loss_gaze = criterion_gaze(predicted_gaze.squeeze(0), target_gaze_embeddings, similarity_target)

            loss_laughter = criterion(logits_laughter.squeeze(0), y_train_fold_laughter)
            loss = loss_gaze + loss_laughter

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate on the test set for the current fold
        model.eval()
        with torch.no_grad():
            (test_logits_gaze, test_logits_laughter) = model([X_test_fold], device)

            similarity_target = torch.ones(test_logits_gaze.size(0), device=device)
            # Get embeddings for target gaze annotations
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            target_gaze_annotations = [gaze_chi_annotations[idx] for idx in y_test_fold_gaze.tolist()]
            target_gaze_embeddings = torch.tensor(model.sentence_encoder.encode(target_gaze_annotations), dtype=torch.float32).to(device)

            test_loss_gaze = criterion_gaze(test_logits_gaze.squeeze(0), target_gaze_embeddings, similarity_target)

            test_loss_laughter = criterion(test_logits_laughter.squeeze(0), y_test_fold_laughter)
            test_total_loss = test_loss_gaze + test_loss_laughter

            # Need to get actual predicted gaze annotations to calculate accuracy, not the raw logits/embeddings
            gaze_chi_annotations = list(vocab_gaze_chi.keys())
            gaze_chi_embeddings = model.sentence_encoder.encode(gaze_chi_annotations)
            gaze_chi_embeddings_tensor = torch.tensor(gaze_chi_embeddings).to(device)

            predicted_gaze_annotations = [
                get_closest_gaze_annotation(pred_embed, gaze_chi_embeddings_tensor, gaze_chi_annotations)
                for pred_embed in test_logits_gaze.squeeze(0)  # Squeeze to remove the extra dim
            ]

            # Convert predicted gaze annotations back to indices for metric calculation
            predicted_gaze_indices = [vocab_gaze_chi.get(anno, vocab_gaze_chi["<UNK>"]) for anno in predicted_gaze_annotations]


            test_accuracy_gaze, _, _ = get_quality_metrics(predicted_gaze_indices, y_test_fold_gaze.cpu().numpy())


            test_pred_laughter = torch.argmax(test_logits_laughter.squeeze(0), dim=1)
            test_correct_laughter = (test_pred_laughter == y_test_fold_laughter).sum().item()
            test_accuracy_laughter = test_correct_laughter / y_test_fold_laughter.size(0)


        test_losses.append(test_total_loss.item())
        test_gaze_accuracies.append(test_accuracy_gaze)
        test_laughter_accuracies.append(test_accuracy_laughter)

        print(f"Fold {fold+1} - Test Loss: {test_total_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Laughter Acc: {test_accuracy_laughter:.4f}")

    # Report average performance across all folds
    print("\n--- Cross-Validation Results ---")
    print(f"Average Test Loss: {np.mean(test_losses):.4f}")
    print(f"Average Test Gaze Accuracy: {np.mean(test_gaze_accuracies):.4f}")
    print(f"Average Test Laughter Accuracy: {np.mean(test_laughter_accuracies):.4f}")

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=False,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

CrossValidation(dataset_params, model_params, train_params)

Starting 10-Fold Cross-Validation...


Cross-Validation Progress:   0%|          | 0/10 [00:00<?, ?it/s]


--- Fold 1/10 ---
Fold 1 - Test Loss: 0.9207, Test Gaze Acc: 0.6426, Test Laughter Acc: 0.4775

--- Fold 2/10 ---
Fold 2 - Test Loss: 0.7340, Test Gaze Acc: 0.6216, Test Laughter Acc: 0.7057

--- Fold 3/10 ---
Fold 3 - Test Loss: 0.9705, Test Gaze Acc: 0.5946, Test Laughter Acc: 0.6426

--- Fold 4/10 ---
Fold 4 - Test Loss: 0.7597, Test Gaze Acc: 0.0300, Test Laughter Acc: 0.8649

--- Fold 5/10 ---
Fold 5 - Test Loss: 0.7597, Test Gaze Acc: 0.3784, Test Laughter Acc: 0.7748

--- Fold 6/10 ---
Fold 6 - Test Loss: 1.0754, Test Gaze Acc: 0.6156, Test Laughter Acc: 0.4955

--- Fold 7/10 ---
Fold 7 - Test Loss: 0.7373, Test Gaze Acc: 0.4398, Test Laughter Acc: 0.8554

--- Fold 8/10 ---
Fold 8 - Test Loss: 0.7065, Test Gaze Acc: 0.3765, Test Laughter Acc: 0.8373

--- Fold 9/10 ---
Fold 9 - Test Loss: 0.7094, Test Gaze Acc: 0.1476, Test Laughter Acc: 0.8373

--- Fold 10/10 ---
Fold 10 - Test Loss: 0.8905, Test Gaze Acc: 0.2289, Test Laughter Acc: 0.6807

--- Cross-Validation Results ---
Aver